## Imports

In [1]:
import resources as r
from resources.deconvoluteFuncs import *
from resources.absCorr import PMCorrect
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output
from ipyfilechooser import FileChooser
from tqdm.notebook import tqdm_notebook


%matplotlib ipympl
# %config InlineBackend.figure_format='svg'

layout = widgets.Layout(width='50%')

## Functions for Lifetimes

In [10]:
def expFunc(t, I0, d, b, τ) -> float:
    return np.add(np.multiply(np.exp(np.subtract(0, np.divide(np.subtract(t, d), τ))), I0), b)

def expFuncWC(t, I0, d, b, τ, c) -> float:
    return np.multiply(expFunc(t, I0, d, b, τ), c)

def expWIRF(t, I0, d, b, τ, c):
    global I0_irf
    global τ_irf
    global b_irf
    global d_irf
    # cList = [c]
    # cList_norm = np.divide(cList, np.linalg.norm(cList))
    # c = cList_norm[0]
    return np.add(expFunc(t, I0_irf, d_irf, b_irf, τ_irf), np.multiply(expFunc(t, I0, d, b, τ), c))

def expWIRFx2(t, I0, d, b, τ_1, c_1, τ_2, c_2):
    global I0_irf
    global τ_irf
    global b_irf
    global d_irf
    # cList = [c_1, c_2]
    # cList_norm = np.divide(cList, np.linalg.norm(cList))
    # c_1, c_2 = cList_norm
    out = np.add(expFunc(t, I0_irf, d_irf, b_irf, τ_irf), np.multiply(expFunc(t, I0, d, b, τ_1), c_1))
    return np.add(out, np.multiply(expFunc(t, I0, d, b, τ_2), c_2))

def expWIRFx3(t, I0, d, b, τ_1, c_1, τ_2, c_2, τ_3, c_3):
    global I0_irf
    global τ_irf
    global b_irf
    global d_irf
    # cList = [c_1, c_2, c_3]
    # cList_norm = np.divide(cList, np.linalg.norm(cList))
    # c_1, c_2, c_3 = cList_norm
    out = np.add(expFunc(t, I0_irf, d_irf, b_irf, τ_irf), np.multiply(expFunc(t, I0, d, b, τ_1), c_1))
    out = np.add(out, np.multiply(expFunc(t, I0, d, b, τ_2), c_2))
    return np.add(out, np.multiply(expFunc(t, I0, d, b, τ_3), c_3))

def expWIRFx4(t, I0, d, b, τ_1, c_1, τ_2, c_2, τ_3, c_3, τ_4, c_4):
    global I0_irf
    global τ_irf
    global b_irf
    global d_irf
    # cList = [c_1, c_2, c_3, c_4]
    # cList_norm = np.divide(cList, np.linalg.norm(cList))
    # c_1, c_2, c_3, c_4 = cList_norm
    out = np.add(expFunc(t, I0_irf, d_irf, b_irf, τ_irf), np.multiply(expFunc(t, I0, d, b, τ_1), c_1))
    out = np.add(out, np.multiply(expFunc(t, I0, d, b, τ_2), c_2))
    out = np.add(out, np.multiply(expFunc(t, I0, d, b, τ_3), c_3))
    return np.add(out, np.multiply(expFunc(t, I0, d, b, τ_4), c_4))

def loadAndCull(fc):
    if fc.selected == None:
        raise Exception('Files not loaded!')

    with open(fc.selected, 'r') as f:
        lines = f.readlines()
    
    binSize = binSize_widg.value
    x = []
    y = []
    startLine = 0
    for line in lines:
        if '#PicoHarp 300' in line:
            startLine = 10
            binSize = int(float(lines[8])*1e3)
    binSize_widg.value = binSize

    for line in lines[startLine:]:
        splitline = line.split()
        if len(splitline) == 1:
            try:
                x += [len(x)*binSize*1e-3]
                y += [int(line)]
            except ValueError:
                pass
    return x, y

def chiSQ(y_obs, y_pred, popt) -> float:
    y_obs = y_obs[30:1024+30]
    y_pred = y_pred[30:1024+30]
    dof = len(y_obs)-len(popt)
    return np.sum(np.divide(np.square(np.subtract(y_obs, y_pred)), y_pred))/dof

def fitFL(plot=True) -> None:
    clear_output()
    maxIter = maxIter_widg.value
    binSize = binSize_widg.value
    expCount = expCount_widg.value

    global I0_irf
    global τ_irf
    global FLFuncList
    global b_irf
    global d_irf
    
    global y_irf
    global y_trf
    global t
    global spectrumObject

    FLFuncList = [expWIRF, expWIRF, expWIRFx2, expWIRFx3, expWIRFx4]

    x_irf_raw, y_irf_raw = loadAndCull(fc_irf)
    x_raw, y_raw = loadAndCull(fc_main)

    x_start = y_raw.index(max(y_raw))+startOffset_widg.value
    
    x = x_raw[x_start:]
    x = np.subtract(x, (x_start*binSize)/1000)
    t = x
    y = y_raw[x_start:]
    y_trf  = y

    x_irf = x_irf_raw[x_start:]
    # x_irf = x_irf_raw
    x_irf = np.subtract(x_irf, (x_start*binSize)/1000)
    y_irf = y_irf_raw[x_start:]
    # y_irf = y_irf_raw
    
    

    if plot:
        fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,6), sharex=True, height_ratios=[3, 1])
        ax1.set_xlim(xRange_widg.value[0], xRange_widg.value[1])
        x_func = np.arange(xRange_widg.value[0], xRange_widg.value[1], 0.01)
        ax1.set_xlabel('Time (ns)')
        ax1.set_ylabel('Count')
        ax1.axvline(0, c='k', lw=0.5)
        ax2.set_xlabel('Time (ns)')
        ax2.set_ylabel('Residuals')
        ax2.set_ylim(-500, 500)
        ax2.axvline(0, c='k', lw=0.5)
        ax2.axhline(0, c='k', lw=0.5)

        ax1.scatter(x_irf , y_irf, c='r', s=0.1)
        ax1.scatter(x, y, s=0.1)

    popt_irf, residual = convolve(x_irf, y_irf, maxIter, True)
    if plot: ax1.plot(x_irf, expFunc(x_irf, *popt_irf), 'b--', lw=0.5)
    I0_irf = popt_irf[0]
    d_irf = popt_irf[1]
    b_irf = popt_irf[2]
    τ_irf = popt_irf[3]


    popt, residual = convolve(x, y, maxIter, False) 
    I0 = popt[0]
    d = popt[1]
    b = popt[2]
    popt_culled = popt[3:]
    trf_fit = []

    cList = []
    for i in range(expCount):
        cList += [popt_culled[(i*2)+1]]
    cList_norm = np.multiply(cList, 1/np.sum(cList))

    I0 = I0/(1/np.sum(cList))

    popt = [I0, d, b]
    for i in range(expCount):
        τ = popt_culled[i*2]
        c = cList_norm[i]
        popt += [τ, c]

    chi2 = chiSQ(y, FLFuncList[expCount](x, *popt), popt)
    print(f'𝜒²: {chi2}')
    print(f'Residual: {residual:.8f}')
    print(f'I0: {I0:.2f}')
    print(f'Baseline: {b:.2f}')
    print(f'Δ: {d:.2f}')


    if plot: 
        for i in range(expCount):
            τ = popt_culled[i*2]
            c = cList_norm[i]
            print(f'τ: {τ:.2f}')
            print(f'Contribution: {c*100:.1f}%')
            trf_fit += [r.trf(τ, c)]
            if scaled_widg.value:
                ax1.plot(x_func, expFuncWC(x_func, I0, d, b, τ, c), 'g--', lw=0.5)
            else:
                ax1.plot(x_func, expFunc(x_func, I0, d, b, τ), 'g--', lw=0.5)
        
        ax1.plot(x_func, FLFuncList[expCount](x_func, *popt), 'k--', lw=1)
        ax1.set_ylim(-200, min(max(y)+300, max(FLFuncList[expCount](x_func, *popt))))
        ax1.scatter([d], [I0], c='purple')
        residualList = np.subtract(FLFuncList[expCount](x, *popt), y_trf)
        ax2.scatter(t, residualList, s=0.1)
        display(plt.show())

        irf_fit = r.irf(I0_irf, d_irf, b_irf, τ_irf)
        spectrumObject = r.Lifetime(r.spectraType.lifetime, y_irf, y_trf, t, binSize, irf_fit, trf_fit, residual, chi2, I0, d, b)

    return residual, chi2, popt

def convolve(x, y, maxIter, irf:bool):
    global FLFuncList
    expCount = expCount_widg.value

    if irf: 
        minbounds = [max(y)-500, -100, -10, 0]
        maxbounds = [max(y)+500,  100, 100, 10]
        popt, pcov = curve_fit(expFunc, x, y, bounds=(minbounds, maxbounds), maxfev=maxIter)
        residual = np.abs(np.sum(np.subtract(y, expFunc(x, *popt))))
    else:
        minbounds = [max(y)-500, -100, -10]
        maxbounds = [max(y)+500,  100, 100]
        for i in range(expCount):
            minbounds += [0,     0]
            maxbounds += [10000, 1]

        popt, pcov = curve_fit(FLFuncList[expCount], x, y, bounds=(minbounds, maxbounds), maxfev=maxIter)

        residual = np.sum(np.subtract(y, FLFuncList[expCount](x, *popt)))
    return popt, residual

def scan(variable):
    scanOffset = scanOffset_widg.value
    expCount = expCount_widg.value
    residualList = []
    offsetList = []

    for offset in range(*scanOffset):
        startOffset_widg.value = offset
        residual, chi2, popt = fitFL(plot=False)
        if variable == 'Residual':
            residualList += [abs(residual)]
        if variable == 'cs':
            if chi2 == np.nan:
                residualList += [99999999999]
            else:
                residualList += [chi2]
        offsetList += [offset]
    best = residualList.index(min(residualList))

    startOffset_widg.value = offsetList[best]
    return

## Functions for Deconvolution

In [3]:
def process(x:list[float], y:list[float], ngauss:int, amp:list[float, float], 
            sigma:list[int, int], maxIter:int) -> tuple[list[float],float,list[float],list[float]]:
    global funcList
    funcList = [gaussian_func, gaussian_func, gaussian_2,  gaussian_3,  gaussian_4,  gaussian_5,  gaussian_6,  gaussian_7,
                gaussian_8,    gaussian_9,    gaussian_10, gaussian_11, gaussian_12, gaussian_13, gaussian_14, gaussian_15,
                gaussian_16,   gaussian_17,   gaussian_18, gaussian_19, gaussian_20]
    
    minbounds = []
    maxbounds = []
    for i in range(ngauss):
        minbounds += [amp[0], min(x), sigma[0]]
        maxbounds += [amp[1], max(x), sigma[1]]

    popt_gauss, pcov_gauss = curve_fit(funcList[ngauss], x, y, bounds=(minbounds, maxbounds), maxfev=maxIter)

    amps = []
    for count, i in enumerate(popt_gauss):
        if count %3 == 0:
            amps += [i]
    
    residual = np.abs(np.sum(np.subtract(y, funcList[ngauss](x, *popt_gauss))))
    return amps, residual, popt_gauss, pcov_gauss

def fit(clip:tuple[float, float], autoClip:bool, amp:tuple[float, float], sigma:tuple[int, int], convergence:float, 
        gaussRange:tuple[int, int], spectraType:r.spectraType, maxIter:int,
        baseLine:str) -> None:
    global fc
    global xin_abs
    global yin_abs
    global xin_fluor
    global yin_fluor
    if fc.selected == None and spectraType == r.spectraType.absorbance:
        x_nm = xin_abs
        y    = yin_abs
    elif fc.selected == None and spectraType == r.spectraType.emission:
        x_nm = xin_fluor
        y    = yin_fluor
    else:
        x_nm, y = loadCSV(fc, clip, spectraType)
    baseLineShift = True if baseLine == 'Shift' else False
    baseLineLevel = True if baseLine == 'Level' else False
    if baseLine != 'None':
        y = baseLineCorrect(y, baseLineShift, baseLineLevel)

    if autoClip: x_nm, y = clipFunc(x_nm, y)

    x = r.nmToEv(x_nm)
    y_norm = np.divide(y, np.linalg.norm(y))

    global spectrumObject
    global funcList
    try:
        del spectrumObject
    except NameError:
        pass

    converged = False
    with tqdm_notebook(total=(gaussRange[1]-gaussRange[0])+1) as pBar:
        for ngauss in range(gaussRange[0], gaussRange[1]):
            amps, residual, popt_gauss, pcov_gauss = process(x, y_norm, ngauss, amp, sigma, maxIter)
            pBar.update(1)
            pBar.set_description(f'Residual: {residual:.6f}')
            conv = 1*10**(-convergence)
            if residual <= conv:
                opt_ngauss = ngauss
                converged = True
                break
        pBar.n = gaussRange[1]-gaussRange[0]
        pBar.update()
    if converged == False:
        print("Not Converged")
        return

    
    amps, residual, popt_gauss, pcov_gauss = process(x, y_norm, opt_ngauss, amp, sigma, maxIter)
    gauss = []
    gaussObjectList = []
    count = 1
    for i in popt_gauss:
        if count != 3:
            gauss += [i]
            count += 1
        else:
            count = 1
            loc = round(gauss[1], 3)
            nm = int(round(r.evToNm(loc), 0))
            
            gaussObjectList += [r.gaussian(loc, i, gauss[0])]
            gauss = []
    
    print(f'Residual: {residual:.2e} ≤ {conv:.2e}')
    print(f'Gaussians: {ngauss}')
    fig, ax = plt.subplots(1,1, figsize=(8,4))
    for i in range(0, ngauss*3, 3):
        ax.fill_between(x, gaussian_func(x, *popt_gauss[i:i+3]), alpha=0.3)
    ax.plot(x, y_norm, 'red', lw=1)
    ax.plot(x, funcList[ngauss](x, *popt_gauss), 'k--', lw=1)
    ax.axhline(0, c='k', lw=0.5)
    # ax.plot(x, np.subtract(0,shiftAmount))
    ax.invert_xaxis()
    ax.set_ylabel("Absorbance/Emission (AU)")
    ax.set_xlabel("λ (nm)")

    params = r.deconvParams(amp, sigma, convergence, gaussRange, maxIter)
    spectrumObject = r.spectrum(spectraType, params, residual, x_nm, y, gaussObjectList)

    if spectraType in [r.spectraType.emission]:
        for gauss in spectrumObject.peaks_sorted_forward:
            print(f'Gauss - loc: {gauss.center:.3f} eV ({r.evToNm(gauss.center)} nm) amp: {gauss.amplitude:.3f}  width: {gauss.width:.3f}')

    if spectraType in [r.spectraType.absorbance, r.spectraType.excitation]:
        for gauss in spectrumObject.peaks_sorted_reverse:
            print(f'Gauss - loc: {gauss.center:.3f} eV ({r.evToNm(gauss.center)} nm) amp: {gauss.amplitude:.3f}  width: {gauss.width:.3f}')
    plt.show()

def loadCSV(fc, clip:tuple[float, float], spectraType:r.spectraType, multi=False) -> tuple[list[float],list[float]] | list[tuple[list[float],list[float]]]:
    file = fc.selected
    if file == None: raise Exception('File not selected')
    x, y = ([], [])
    if multi: 
        x1, y1, x2, y2, x3, y3, x4, y4, x5, y5 = ([], [], [], [], [], [], [], [], [], [])
        varListList = [x1, y1, x2, y2, x3, y3, x4, y4, x5, y5]
        varTupList = [(x1, y1), (x2, y2), (x3, y3), (x4, y4), (x5, y5)]
    printedNM = False
    with open(file, 'r') as f:
        lines = f.readlines()
    for line in lines:
        CSV = True if ',' in line else False
        if CSV:
            splitline = line.split(',')
        else:
            splitline = line.split()

        if 'Ex. Wavelength (nm)' in line and not printedNM:
            global csvExLambda
            csvExLambda = int(round(float(line.split()[-1]), 0))
            exLambda_widg.value = csvExLambda
            printedNM = True

        try:
            if not multi:
                if len(splitline) != 1:
                    x_temp, y_temp = [float(i) for i in splitline[0:2]]
                    x += [x_temp]
                    y += [y_temp]

            if multi:
                if len(splitline) in [10, 11]:
                    x1_temp, y1_temp, x2_temp, y2_temp, x3_temp, y3_temp, x4_temp, y4_temp, x5_temp, y5_temp = [float(i) for i in splitline[0:10]]
                    tempVarList = [x1_temp, y1_temp, x2_temp, y2_temp, x3_temp, y3_temp, x4_temp, y4_temp, x5_temp, y5_temp]
                    for i, j in zip(varListList, tempVarList): i += [j]

        except ValueError:
            pass
        except IndexError:
            pass

    if not multi:
        x_ev = r.nmToEv(x)
        less = np.less(x_ev, clip[1])
        more = np.greater(x_ev, clip[0])
        xor = np.logical_xor(less, more)
        x = np.delete(x, xor)
        y = np.delete(y, xor)
        if spectraType in [r.spectraType.emission, r.spectraType.excitation]:
            y = PMCorrect(x, y)
        return x, y

    if multi:
        outTupList = []
        for x, y in varTupList:
            x_ev = r.nmToEv(x)
            less = np.less(x_ev, clip[1])
            more = np.greater(x_ev, clip[0])
            xor = np.logical_xor(less, more)
            x = np.delete(x, xor)
            y = np.delete(y, xor)
            y = PMCorrect(x, y)
            outTupList += [(x, y)]
        return outTupList


def baseLineCorrect(y, shift:bool, level:bool):
    if shift:
        y = np.subtract(y,min(y))
        shiftAmount = [min(y) for i in range(len(y))]
    
    if level:
        y = np.subtract(y,min(y))
        maxLoc = list(y).index(max(y))
        lower = y[0:maxLoc]
        lowerLoc = list(lower).index(min(lower))
        upper = y[maxLoc:-1]
        upperLoc = list(upper).index(min(upper))
        
        innerRange = (upperLoc+len(lower))-lowerLoc
        interval = ((min(lower)-min(upper))/innerRange)
        shiftAmount = [i*interval for i in range(1,len(y)+1)]
        shiftAmount = np.subtract(shiftAmount, (len(y)-upperLoc)*interval)
        y = np.add(y, shiftAmount)
        y = np.subtract(y,min(y))

    return y

def clipFunc(x:list[float], y:list[float], units:str='eV' ) -> tuple[list[float],list[float]]:
    maxLoc = list(y).index(max(y))
    lower = y[0:maxLoc]
    lowerLoc = list(lower).index(min(lower))
    upper = y[maxLoc:-1]
    upperLoc = list(upper).index(min(upper))

    less = np.less(x, x[upperLoc+len(lower)])
    more = np.greater(x, x[lowerLoc])
    if units == 'eV': print(f'Clipped from {r.nmToEv(x[upperLoc+len(lower)]):.3f} eV to {r.nmToEv(x[lowerLoc]):.3f} eV')
    if units == 'nm': print(f'Clipped from {x[upperLoc+len(lower)]:.0f} nm to {x[lowerLoc]:.0f} nm')
    xor = np.logical_xor(less, more)
    x = np.delete(x, xor)
    y = np.delete(y, xor)
    return x, y

In [4]:
global fc
fc = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/')
fc.filter_pattern = '*.csv'


def ftir() -> None:
    global spectrumObject
    try:
        del spectrumObject
    except NameError:
        pass
    x, y = loadCSV()
    spectrumObject = r.spectrum(r.spectraType.ftir, None, None, x, y, None)
    print('Loaded!')

# display(widgets.interactive(ftir, {'manual' : True, 'manual_name' : 'Load FTIR'}))

s10_abs_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/abs')
s10_abs_widg.title = '0.1 AU Absorbance csv'
s10_abs_widg.filter_pattern = '*.csv'
s08_abs_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/abs')
s08_abs_widg.title = '0.08 AU Absorbance csv'
s08_abs_widg.filter_pattern = '*.csv'
s06_abs_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/abs')
s06_abs_widg.title = '0.06 AU Absorbance csv'
s06_abs_widg.filter_pattern = '*.csv'
s04_abs_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/abs')
s04_abs_widg.title = '0.04 AU Absorbance csv'
s04_abs_widg.filter_pattern = '*.csv'
s02_abs_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/abs')
s02_abs_widg.title = '0.02 AU Absorbance csv'
s02_abs_widg.filter_pattern = '*.csv'
abs_widg = widgets.HBox([s10_abs_widg, s08_abs_widg, s06_abs_widg, s04_abs_widg, s02_abs_widg])
abs_composite_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/abs')
abs_composite_widg.title = 'Composite Absorbance csv'
abs_composite_widg.filter_pattern = '*.csv'

children = [abs_composite_widg, abs_widg]
tab_abs = widgets.Tab()
tab_abs.children = children
tab_abs.set_title(0, 'Composite')
tab_abs.set_title(1, 'Individual')

s10_fluo_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/fluor')
s10_fluo_widg.title = '0.1 AU Fluorescence csv'
s10_fluo_widg.filter_pattern = '*.csv'
s08_fluo_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/fluor')
s08_fluo_widg.title = '0.08 AU Fluorescence csv'
s08_fluo_widg.filter_pattern = '*.csv'
s06_fluo_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/fluor')
s06_fluo_widg.title = '0.06 AU Fluorescence csv'
s06_fluo_widg.filter_pattern = '*.csv'
s04_fluo_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/fluor')
s04_fluo_widg.title = '0.04 AU Fluorescence csv'
s04_fluo_widg.filter_pattern = '*.csv'
s02_fluo_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/fluor')
s02_fluo_widg.title = '0.02 AU Fluorescence csv'
s02_fluo_widg.filter_pattern = '*.csv'
fluo_widg = widgets.HBox([s10_fluo_widg, s08_fluo_widg, s06_fluo_widg, s04_fluo_widg, s02_fluo_widg])
fluo_composite_widg = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/fluor')
fluo_composite_widg.title = 'Composite Fluorescence csv'
fluo_composite_widg.filter_pattern = '*.csv'

children = [fluo_composite_widg, fluo_widg]
tab_fluo = widgets.Tab()
tab_fluo.children = children
tab_fluo.set_title(0, 'Composite')
tab_fluo.set_title(1, 'Individual')

def processBasicSpectra(fc:FileChooser, clip:tuple[int, int], autoClip:bool, baseLine:str, multi:bool=False) -> r._simpleSpectrum | list[r._simpleSpectrum]:
    clip = (r.nmToEv(clip[0]), r.nmToEv(clip[1]))
    tupList = loadCSV(fc, clip, r.spectraType.qy, multi=multi)
    outList = []
    if not multi: tupList = [tupList]
    for x, y in tupList:
        baseLineShift = True if baseLine == 'Shift' else False
        baseLineLevel = True if baseLine == 'Level' else False
        if baseLine != 'None':
            y = baseLineCorrect(y, baseLineShift, baseLineLevel)

        if autoClip: x, y = clipFunc(x, y, units='nm')
        integrand = np.abs(np.trapz(y, x=x))

        if not multi:
            return r._simpleSpectrum(maxy=max(y), integrand=integrand, x=x, y=y)
        else:
            outList += [r._simpleSpectrum(maxy=max(y), integrand=integrand, x=x, y=y)]
    return outList

def loadSeries(absClip:tuple[int, int], fluoClip:tuple[int, int], autoClip:bool, baseLine:str) -> None:
    global spectrumObject

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10,4))
    ax1.set_ylabel("Absorbance (AU)")
    ax1.set_xlabel("λ (nm)")
    ax1.axhline(0, c='k', lw=0.5)
    if abs_composite_widg.selected == None:
        absSpectraList = [] 
        for inCSV in [s10_abs_widg, s08_abs_widg, s06_abs_widg, s04_abs_widg, s02_abs_widg]:
            absSpectraList += [processBasicSpectra(inCSV, absClip, autoClip, baseLine)]
            ax1.plot(absSpectraList[-1].x, absSpectraList[-1].y)
    else:
        absSpectraList = processBasicSpectra(abs_composite_widg, absClip, autoClip, baseLine, multi=True)
        for spectrum in absSpectraList:
            ax1.plot(spectrum.x, spectrum.y)
            

    ax2.set_ylabel("Emission (AU)")
    ax2.set_xlabel("λ (nm)")
    ax2.axhline(0, c='k', lw=0.5)
    if fluo_composite_widg.selected == None:
        fluoSpectraList = [] 
        for inCSV in [s10_fluo_widg, s08_fluo_widg, s06_fluo_widg, s04_fluo_widg, s02_fluo_widg]:
            fluoSpectraList += [processBasicSpectra(inCSV, fluoClip, autoClip, baseLine)]
            ax2.plot(fluoSpectraList[-1].x, fluoSpectraList[-1].y)
    else:
        fluoSpectraList = processBasicSpectra(fluo_composite_widg, fluoClip, autoClip, baseLine, multi=True)
        for spectrum in fluoSpectraList:
            ax2.plot(spectrum.x, spectrum.y)
    plt.show()
    spectrumObject = r.spectrumSeries(r.spectraType.qy, absSpectraList, fluoSpectraList, None, None)

loadSeries_widg = widgets.interactive(loadSeries, {'manual' : True, 'manual_name' : 'Load QY Series'},
                    absClip = widgets.IntRangeSlider(min=300, max=800, value=[300, 800], description='Absorbance Spectra Clip', orientation='horizontal', readout=True, layout=layout),
                    fluoClip = widgets.IntRangeSlider(min=300, max=800, value=[300, 800], description='Fluorescence Spectra Clip', orientation='horizontal', readout=True, layout=layout),
                    autoClip = widgets.Checkbox(value=False, description='Auto Clip the Spectrum', layout=layout),
                    baseLine = widgets.ToggleButtons(options = ['Shift', 'Level', 'None'], value='None', description="Baseline Correction"))

fluorophores, solvents, methods = r.fluorophores_solvents_methods()

def saveSpectrum() -> None:
    global saveFluorophore
    global saveSolvent
    global litQY
    global exLambda
    global spectrumObject
    global useCSV
    global csvExLambda
    if spectrumObject.spectrum == r.spectraType.qy:
        spectrumObject.qy = litQY
        spectrumObject.excitation = exLambda
        if useCSV: spectrumObject.excitation = csvExLambda

    if saveFluorophore in fluorophores and saveSolvent in solvents:
        try:
            with r.statusLoad('spectra') as df:
                df.at[(saveFluorophore, spectrumObject.spectrum), saveSolvent] = spectrumObject
                print(f'Saved {saveFluorophore} in {saveSolvent}!')
        except NameError:
            print('Spectrum not loaded/deconvoluted yet!')
    else:
        print(f'{saveFluorophore} in {saveSolvent} isn\'t in the main dataset!')

def saveRefSpectrum() -> None:
    global saveFluorophore
    global saveSolvent
    global useStored
    global litQY
    global exLambda
    global spectrumObject
    global useCSV
    global csvExLambda
    if spectrumObject.spectrum == r.spectraType.qy:
        spectrumObject.qy = litQY
        spectrumObject.excitation = exLambda
        if useCSV: spectrumObject.excitation = csvExLambda

    try:
        with r.statusLoad('qy_ref') as df:
            df.at[(saveFluorophore, spectrumObject.spectrum), saveSolvent] = spectrumObject
            print(f'Saved {saveFluorophore} in {saveSolvent}!')
    except NameError:
        print('Spectrum not loaded/deconvoluted yet!')

def showDS() -> None:
    with r.statusLoad('spectra') as df:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(df.notnull().style.applymap(lambda x: 'color : blue' if x == True  else 'color : red'))

    with r.statusLoad('qy_ref') as df:
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(df.notnull().style.applymap(lambda x: 'color : blue' if x == True  else 'color : red'))

fit_widg = widgets.interactive(fit, {'manual' : True, 'manual_name' : 'Deconvolute'},
                    clip = widgets.FloatRangeSlider(min=0, max=5, step=0.001, value=[0, 5], description='Spectra Range (Manual Clipping)', orientation='horizontal', readout=True, layout=layout),
                    autoClip = widgets.Checkbox(value=True, description='Auto Clip the Spectrum', layout=layout),
                    amp = widgets.FloatRangeSlider(value=[0.00, 0.5], min=0, max=0.6, step=0.01, description='Amplitude Range', orientation='horizontal', readout=True, layout=layout),
                    sigma = widgets.FloatRangeSlider(value=[0, 1], min=0, max=2, step=0.01, description='Width Range', orientation='horizontal', readout=True, layout=layout),
                    gaussRange = widgets.IntRangeSlider(value=[1, 5], min=1, max=20, step=1, description='Range of Gaussians to Test', orientation='horizontal', readout=True, layout=layout),
                    convergence =  widgets.FloatSlider(value=2.2, min=0., max=10., step=0.1, description=r'Convergence (1e-n)', layout=layout),
                    maxIter = widgets.IntText(value=5000,  step=1000, description='Maximum fitting iterations'),
                    baseLine = widgets.ToggleButtons(options = ['Shift', 'Level', 'None'], value='Level', description="Baseline Correction"),
                    spectraType = widgets.ToggleButtons(options = [r.spectraType.absorbance, r.spectraType.emission, r.spectraType.excitation], description="Spectra Type"),
                    )

fluorophore_widg = widgets.Dropdown(options=r.Fluorophores, description='Fluorophore')
solvent_widg = widgets.Dropdown(options=r.Solvents, description='Solvent')
useStored_widg = widgets.Checkbox(value=False, description='Use Φ and λΦ from fluorophore', layout=layout)
useCSV_widg = widgets.Checkbox(value=True, description='Use λΦ from CSV File', layout=layout)
litQY_widg = widgets.BoundedFloatText(value=0.00, min=0.00, max=1, step=0.01, description='Literature Φ for use as Std.')
exLambda_widg = widgets.IntText(min=200, max=800, description='Excitation Wavelength (nm)')

binSize_widg = widgets.BoundedIntText(value=8, min=0, max=512, description="Bin Size (ps) (If not in file)")
maxIter_widg = widgets.BoundedIntText(value=5000, min=0, max=15000, step=1000, description="Maximum Fitting Iterations")
expCount_widg = widgets.ToggleButtons(options=[1, 2, 3, 4], value=1, description="Number of exponentials to Fit")
xRange_widg = widgets.FloatRangeSlider(value=[-2, 35], min=-5, max=50, description='X axis range', layout=layout)
startOffset_widg = widgets.BoundedIntText(value=-20, min=-300, max=300, description="Offset for the Starting Bin")
scaled_widg = widgets.Checkbox(value=True, description="Plot the Decays Scaled by Their Coefficients", layout=layout)
scanOffset_widg = widgets.IntRangeSlider(value=[-20,10], min=-100, max=50, description="Scan Offset for the Starting Bin", layout=layout)


def saveLoader(fluorophore_in, solvent_in, useStored_in, litQY_in, exLambda_in, maxIter_in, binSize_in, expCount_in, startOffset_in, scanOffset_in, useCSV_in, xRange_in) -> None:
    global saveFluorophore
    global saveSolvent
    global useStored
    global litQY
    global exLambda
    global maxIter
    global binSize
    global expCount
    global startOffset
    global scanOffset
    global useCSV
    global xRange

    maxIter = maxIter_in
    binSize = binSize_in
    expCount = expCount_in
    startOffset = startOffset_in
    scanOffset = scanOffset_in
    xRange = xRange_in

    
    saveFluorophore = fluorophore_in
    saveSolvent = solvent_in
    useStored = useStored_in
    useCSV = useCSV_in
    if useStored and saveSolvent == saveFluorophore.qysolvent:
        exLambda_widg.value = saveFluorophore.qyLambda
        exLambda = saveFluorophore.qyLambda
        litQY_widg.value = saveFluorophore.qy
        litQY = saveFluorophore.qy
    elif not useStored:
        exLambda = exLambda_in
        litQY = litQY_in

saveLoader_widg = widgets.interactive_output(saveLoader, {'fluorophore_in': fluorophore_widg, 
                                                          'solvent_in': solvent_widg, 
                                                          'useStored_in': useStored_widg, 
                                                          'litQY_in': litQY_widg, 
                                                          'exLambda_in': exLambda_widg,
                                                          'binSize_in': binSize_widg,
                                                          'maxIter_in': maxIter_widg, 
                                                          'expCount_in': expCount_widg, 
                                                          'startOffset_in': startOffset_widg,
                                                          'scanOffset_in': scanOffset_widg,
                                                          'useCSV_in': useCSV_widg,
                                                          'xRange_in': xRange_widg})

save_widg = widgets.interactive(saveSpectrum, {'manual' : True, 'manual_name' : 'Save Spectra'})
save_ref_widg = widgets.interactive(saveRefSpectrum, {'manual' : True, 'manual_name' : 'Save Ref Spectra'})

showDS_widg = widgets.interactive(showDS, {'manual' : True, 'manual_name' : 'Show DS'})

def fromQY(fluorophore:r.Fluorophores, solvent:r.Solvents):
    global xin_abs
    global yin_abs
    global xin_fluor
    global yin_fluor
    with r.statusLoad('spectra') as df:
        with r.statusLoad('qy_ref') as df_qy:
            try:
                spectrum = df.at[(fluorophore, r.spectraType.qy), solvent]
                if spectrum != None:
                    xin_abs = spectrum.absorbanceSpectra[0].x
                    yin_abs = spectrum.absorbanceSpectra[0].y
                    xin_fluor = spectrum.emissionSpectra[0].x
                    yin_fluor = spectrum.emissionSpectra[0].y
                    print('Loaded!')
                else:
                    print('Spectrum not imported')
            except KeyError:
                try:
                    spectrum = df_qy.at[(fluorophore, r.spectraType.qy), solvent]
                    if spectrum != None:
                        xin_abs = spectrum.absorbanceSpectra[0].x
                        yin_abs = spectrum.absorbanceSpectra[0].y
                        xin_fluor = spectrum.emissionSpectra[0].x
                        yin_fluor = spectrum.emissionSpectra[0].y
                        print('Loaded!')
                    else:
                        print('Spectrum not imported')
                except KeyError:
                    print('Fluorophore/Solvent combo not found')
    return

fromQY_widg = widgets.interactive(fromQY, 
                    fluorophore=widgets.Dropdown(options=r.Fluorophores, description='Fluorophore'),
                    solvent=widgets.Dropdown(options=r.Solvents, value=r.Solvents.etoh, description='Solvent'))

children = [fc, fromQY_widg]
loadTab = widgets.Tab()
loadTab.children = children
loadTab.set_title(0, 'Load From File')
loadTab.set_title(1, 'Load From QY Spectra')



fc_main = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/tr')
fc_main.filter_pattern = '*.txt'
fc_main.title = 'Select TRF'

fc_irf = FileChooser('/Users/adrea/gdrive/Monash/PhD/Fluorophore/data/tr')
fc_irf.filter_pattern = '*.txt'
fc_irf.title = 'Select IRF'

scan_widg = widgets.interactive(scan, {'manual' : True, 'manual_name' : 'Minimise function'},
                                    variable=widgets.ToggleButtons(options={'Residual': 'r', '𝜒²': 'cs'}, value='r', description='Variable to Minimise for'))

fitFL_widg = widgets.interactive(fitFL, {'manual' : True, 'manual_name' : 'Plot'}, plot=widgets.Checkbox(value=True, layout=widgets.Layout(display = "none")))

deconvolute_widg = widgets.VBox([loadTab,
                                 fit_widg, 
                                 fluorophore_widg, solvent_widg,
                                 save_widg, 
                                 showDS_widg])

qy_widg = widgets.VBox([tab_abs, 
                        tab_fluo, 
                        loadSeries_widg,
                        fluorophore_widg, solvent_widg, useStored_widg, useCSV_widg, litQY_widg, exLambda_widg,
                        saveLoader_widg, save_widg, save_ref_widg,
                        showDS_widg])

lifetime_widg = widgets.VBox([fc_main,
                              fc_irf,
                              binSize_widg, 
                              maxIter_widg, 
                              expCount_widg, 
                              startOffset_widg,
                              scanOffset_widg,
                              xRange_widg,
                              scaled_widg,
                              saveLoader_widg,
                              scan_widg,
                              fitFL_widg,
                              fluorophore_widg, solvent_widg, saveLoader_widg, save_widg,
                              showDS_widg])

children = [deconvolute_widg, qy_widg, lifetime_widg]
tab = widgets.Tab()
tab.children = children
tab.set_title(0, 'Deconvolute')
tab.set_title(1, 'Quantum Yield')
tab.set_title(2, 'Fluorescence Lifetime')
display(tab)

display(HTML('''<style>
    .widget-label { min-width: 30ex !important; }
    .widget-button { min-width: max-content }
</style>'''))


FL uses the form of the function:

$$
fl(t)=I_0 exp\bigg(-\frac{t-b}{\tau}\bigg)+\Delta
$$

Where:
* $t =$ Time
* $b =$ Up/down baseline shift
* $\Delta =$ Left/right shift
* $\tau =$ Fluorescence lifetime
* $I_0 =$ Initial photon concentration

For the corrected fitting procedure, the convolved function is calculated as:

$$
fl=fl(t)_{IRF}+\sum_{i=1}^{\text{Exponents}} C_i\cdot fl(t)_i
$$

Where:
* $fl(t)_{IRF} =$ Fitted instrument response function
* $fl(t)_i =$ Fitted function for each decay
* $C_i =$ Linear coefficient for each decay

While not explicitly programmed, $C_i$ should theoretically sum to $1$
